#Vertex Agents SDK for quering MongoDB Atlas

This colab demonstrates all how to use Vertex AI extension to query MongoDB Atlas

##Authenticate

In [2]:
from google.colab import auth
auth.authenticate_user()

##Download most recent copy of Vertex Agents SDK

In [3]:
!gsutil cp gs://vertex_agents_private_releases/vertex_agents/google_cloud_aiplatform-1.60.dev20240710+vertex.agents-py2.py3-none-any.whl .

Copying gs://vertex_agents_private_releases/vertex_agents/google_cloud_aiplatform-1.60.dev20240710+vertex.agents-py2.py3-none-any.whl...
/ [1 files][  5.2 MiB/  5.2 MiB]                                                
Operation completed over 1 objects/5.2 MiB.                                      


In [4]:
!gsutil cp gs://vertex_agents_private_releases/vertex_agents/google_cloud_aiplatform-1.55.dev20240609+vertex.agents-py2.py3-none-any.whl .

!pip install --upgrade --force-reinstall -q google_cloud_aiplatform-1.55.dev20240609+vertex.agents-py2.py3-none-any.whl --no-warn-conflicts
!pip install numpy==1.26.4

# Restart the kernel runtime to load the private preview SDK
import IPython
app = IPython.Application.instance()
app.kernel.do_shutdown(True)

Copying gs://vertex_agents_private_releases/vertex_agents/google_cloud_aiplatform-1.55.dev20240609+vertex.agents-py2.py3-none-any.whl...
/ [1 files][  5.1 MiB/  5.1 MiB]                                                
Operation completed over 1 objects/5.1 MiB.                                      
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.6/294.6 kB 6.2 MB/s eta 0:00:00
  Using cached numpy-1.26.4-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (61 kB)
Using cached numpy-1.26.4-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (18.2 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 2.1.1
    Uninstalling numpy-2.1.1:
      Successfully uninstalled numpy-2.1.1


{'status': 'ok', 'restart': True}

##Imports and Initialization

Make sure to initialize aiplatform with your projectID, location, and api endpoint. You need to initialize aiplatform before performing any of the other imports

In [1]:
# TODO: Replace with your project ID.
PROJECT_ID = "gcp-pov" # @param {type:"string"}
LOCATION = 'us-central1' # @param {type:"string"}
STAGING_BUCKET =  "gs://vertexai_extensions" # @param {type:"string"}
API_ENDPOINT = 'us-central1-aiplatform.googleapis.com' # @param {type:"string"}
LLM_MODEL = "gemini-1.5-pro" # @param {type:"string"}

from google.cloud import aiplatform
from google.cloud.aiplatform.private_preview.vertex_agents.app import App, Session
from google.cloud.aiplatform.private_preview.vertex_agents.agent import Agent
from vertexai.preview.extensions import Extension

aiplatform.init(project=PROJECT_ID, location=LOCATION, api_endpoint=API_ENDPOINT)

#Apps

##Create App

There are two options for creating an app.


1.   use the App.create function, which will create a new app for you
2.   initialize a Vertex Agents App object with the resource name of a previously created App



In [2]:
app = App.create(display_name='Vertex Agents App for MongoDB', description='made via the SDK')

# To load an existing app
# app = App("projects/<project_id>/locations<location_id>/apps/<app_id>")

##List Apps

You can list all the apps in a project, which will return a list of operational Apps.

Alternatively, you can get a user-friendly mapping of App display names and resource names.

In [3]:
all_apps = App.list_apps()
all_apps[0]

app_name: projects/787220387490/locations/us-central1/apps/4618722892845154304
display_name: Vertex Agents App for MongoDB
description: made via the SDK
agents: {}

# Tools

###Extension Tools

####MongoDB Extensions

Run the 1. extensions for MongoDB notebook to generate the extension id and provide to EXTENSIONS_ID variable.

In [8]:
from google.cloud import aiplatform
from vertexai.preview import extensions

EXTENSIONS_ID="projects/787220387490/locations/us-central1/extensions/4347468976226304000"

mdb_extension = extensions.Extension(EXTENSIONS_ID)

# Operations on an app

## Agents

Agents exist under an App. Agents can perform small, specific tasks utilizing Vertex Extensions and function calling. Agents require a display name, and a set of instructions.

When creating an agent, you must specify the following:


1.   Display Name
2.   Instructions - the detailed instructions the Agent should follow. This is where you instruct your agent when to call your functions / extensions and with what parameters.

You also have the option to specify the extensions and functions you will be using. Please note that if you reference an extension or function in the instructions, you ***must*** include an extension/function with the same display name in the extensions/functions lists.

##Create Agent
You can create an agent for your app by using the add_agent method. When creating an agent, you must specify all the required fields discussed above.



##Create a single Agent App

If you want to create an App and Agent in a single turn (i.e. avoiding using create_app -> add_agent) you can use `App.create_single_agent_app`. You must specify the agent display name, and then optionally specify tools for the agent, and the App's display name and description. This function returns an operable App.

In [16]:
DISPLAY_NAME = "MongoDB Agent"
INSTRUCTIONS = """
If user ask about anything from papers published use the extension MongoDb Vertex API Interpreter to make the call,
The agent makes api call to mongodb cluster named 'FreeTier' database sample_papers and collection embedding_papers to do all crud operations only return the documents from the same.
Only respond based on the response of the tool. Do not create your own answers.
embedding_papers schema exaple looks like this : {"_id":{"$oid":"66bf0251e4f2221967175969"},"id":{"$numberDouble":"704.0003"},"submitter":"Hongjun Pan","authors":"Hongjun Pan","title":"The evolution of the Earth-Moon system based on the dark matter field\n  fluid model","comments":"23 pages, 3 figures","journal-ref":null,"doi":null,"report-no":null,"categories":"physics.gen-ph","license":null,"abstract":"  The evolution of Earth-Moon system is described by the dark matter field\nfluid model proposed in the Meeting of Division of Particle and Field 2004,\nAmerican Physical Society. The current behavior of the Earth-Moon system agrees\nwith this model very well and the general pattern of the evolution of the\nMoon-Earth system described by this model agrees with geological and fossil\nevidence. The closest distance of the Moon to Earth was about 259000 km at 4.5\nbillion years ago, which is far beyond the Roche's limit. The result suggests\nthat the tidal friction may not be the primary cause for the evolution of the\nEarth-Moon system. The average dark matter field fluid constant derived from\nEarth-Moon system data is 4.39 x 10^(-22) s^(-1)m^(-1). This model predicts\nthat the Mars's rotation is also slowing with the angular acceleration rate\nabout -4.38 x 10^(-22) rad s^(-2).\n","versions":[{"version":"v1","created":"Sun, 1 Apr 2007 20:46:54 GMT"},{"version":"v2","created":"Sat, 8 Dec 2007 23:47:24 GMT"},{"version":"v3","created":"Sun, 13 Jan 2008 00:36:28 GMT"}],"update_date":{"$date":{"$numberLong":"1200182400000"}},"authors_parsed":[["Pan","Hongjun",""]]}
if the schema mathces to any of the synonyms then return the document.
"""

app = App.create_single_agent_app(
    agent_display_name=DISPLAY_NAME,
    instructions=INSTRUCTIONS,
    extensions={
        'mongodb': mdb_extension
    }
    )

In [ ]:
# DISPLAY_NAME = "MDB Agent"
# INSTRUCTIONS = """
# If user ask about anything from papers published use the extension MongoDb Vertex API Interpreter to make the call,
# The extension makes api call to mongodb cluster FreeTier database sample_papers and collection embedding_papers to do all crud operations only return the documents from the same.
# Only respond based on the response of the tool. Do not create your own answers.
# """

# updated_dinner_agent = created_agent.update(
#     new_display_name=DISPLAY_NAME,
#     new_instructions=INSTRUCTIONS,
#     new_extensions={
#         'mongodb': mdb_crud
#     }
# )

##List Agents

You can list all the agents exiting under an App. This will return operatable Agent objects.

In [ ]:
app.list_agents()

##Get Agent

You can also get a specific Agent using either the resource name or the display name

In [18]:
agent = app.get_agent('MongoDB Agent')

In [ ]:
agent

Alternatively, you can get an Agent using the Agent class

Note that the only way to create a new Agent is to create it under an App, using `app.add_agent()`. The Agents constructor can only be used to get a reference to an Agent that has already been created.

#Session

NOTE: you can remove `.content` from the end of each run call to get more detailed information about each session action

In [29]:
session = app.start_session()

## Multi-turn queries without tool calls


In [ ]:
session.run("Find the books submitted by Pavel Nadolsky").content

In [ ]:
session.get_history()

name: "projects/787220387490/locations/us-central1/apps/7451205583484485632/sessions/467402942723391488"
actions {
  message {
    role: "user"
    parts {
      text: "can you repeat my word"
    }
  }
  create_time {
    seconds: 1719299849
    nanos: 818392000
  }
  turn: 1
}
actions {
  message {
    role: "model"
    parts {
      text: "I am sorry, I cannot fulfill this request. The available tools lack the desired functionality. \n"
    }
  }
  create_time {
    seconds: 1719299849
    nanos: 818392000
  }
  turn: 1
}
actions {
  message {
    role: "user"
    parts {
      text: "Find the title by submitter \'Pavel Nadolsky\'"
    }
  }
  create_time {
    seconds: 1719299889
    nanos: 835159000
  }
  turn: 2
}
actions {
  tool_use {
    extension_invocation {
      extension_name: "projects/787220387490/locations/us-central1/extensions/1815196940434931712"
      operation_id: "findmany_mdb"
    }
    action: "findmany_mdb"
    input_parameters {
      fields {
        key: "f

In [ ]:
session.run("Find the books submitted by Pavel Nadolsky").content

session: "projects/787220387490/locations/us-central1/apps/2172423870252843008/sessions/8022472792614109184"
content {
  role: "model"
  parts {
    text: "Hi! 👋  How can I help you today? 😊 \n"
  }
}
actions {
  message {
    role: "user"
    parts {
      text: "hi"
    }
  }
  create_time {
    seconds: 1723799935
    nanos: 947990000
  }
  turn: 1
}
actions {
  message {
    role: "model"
    parts {
      text: "Hi! 👋  How can I help you today? 😊 \n"
    }
  }
  create_time {
    seconds: 1723799935
    nanos: 947990000
  }
  turn: 1
}

## User query that invokes client side function


In [ ]:
session.run("get me some borough for cuisine American").content

In [ ]:
# app.delete_session('<Session Name>')